# Programmation orientée objet

Comment créer se propres objets en Python. 

## Vocabulaire

En python tout ce que l'on manipule est un objet : 
* Nombre, liste, dictionnaire, ...
* Tableau numpy
* Fonction, module, ....

Un objet possède des **attributs**. Exemple : 

    z = 1.1J
    z.real
    a = np.array([1, 2, 4])
    a.shape
    
Un objet possède des **méthodes**. Exemple :

    a.mean()
    a.conjugate()
    z.conjugate()
    
Les méthodes sont des fonctions attachées à l'objet. La méthode conjugate d'un tableau n'est pas la même que la méthode conjugate d'un complexe

## Comment créer un objet

### Créer un nombre complexe


In [126]:
from math import sqrt, atan2, cos, sin, pi

class Complex:
    def __init__(self, partie_reelle, partie_imaginaire=0):
        self.re = partie_reelle
        self.im = partie_imaginaire
        
    def conjugate(self):
        return Complex(self.re, -self.im)
    
    def __str__(self):
        if self.im==0:
            return f'{self.re}'
        if self.im>0:
            return f'{self.re} + {self.im}J'
        if self.im<0:
            return f'{self.re} - {-self.im}J'
        
    def __repr__(self):
        return f'Complex({self.re}, {self.im})'
    
    def _make_other_complex(self, other):
        if isinstance(other, (int, float)):
            return Complex(other, 0)
        return other
    
    def __add__(self, other):
        other = self._make_other_complex(other)
        if isinstance(other, Complex):
            return Complex(self.re + other.re, self.im + other.im)
        return NotImplemented

    def __sub__(self, other):
        other = self._make_other_complex(other)
        if isinstance(other, Complex):
            return Complex(self.re - other.re, self.im - other.im)
        return NotImplemented
    
    def __truediv__(self, other):
        other = self._make_other_complex(other)
        if isinstance(other, Complex):
            return Complex.from_polar(self.r/other.r, self.phase - other.phase)
        return NotImplemented
            

    def __eq__(self, other):
        other = self._make_other_complex(other)
        if isinstance(other, Complex):
            return other.re==self.re and other.im==self.im
        return NotImplemented
    
    def __radd__(self, other):
        return self + other
    
    @property
    def r(self):
        return sqrt(self.re**2 + self.im**2)
    
    @property
    def phase(self):
        return atan2(self.im, self.re)

    @classmethod
    def from_polar(cls, r, phase):
        return Complex(r*cos(phase), r*sin(phase))

In [127]:
z1 = Complex(1, 0.5)
z2 = Complex(2.3, 1.3)

z1.phase

0.4636476090008061

In [128]:
z1/z2

Complex(0.42263610315186245, -0.021489971346704898)

In [96]:
z = Complex(1, 0.5)

print(z)

1 + 0.5J


In [68]:
z_conj = z.conjugate()
z_conj.im
print(z_conj)

1 - 0.5J


In [69]:
print(z)
str(z)

1 + 0.5J


'1 + 0.5J'

In [71]:
isinstance(z, object)

True

In [55]:
repr(z)

'Complex(1, 0.5)'

In [20]:
s = 'Bonjour'
print(s)

Bonjour


In [21]:
s

'Bonjour'

In [26]:
import numpy as np
l = ['Bonjour, Hello', np.array([1, 2])]

print(l)

['Bonjour, Hello', array([1, 2])]


In [24]:
print(l)

[1, 2]


### Dataclass

In [13]:
import dataclasses

@dataclasses.dataclass
class Book():
    titre : str
    auteur : str
    annee : int
        
book1 = Book("A very nice book", "F. Dupont", 2014)
book1

Book(titre='A very nice book', auteur='F. Dupont', annee=2014)

### Instrumentation

Exemple : oscilloscope (c.f. TP)

Inconvénients de la méthode fonctionnelle : 

* L'utilisateur doit importer beaucoup de fonction
* Les fonctions dépendent de l'instrument, il faut donc connaitre l'instrument pour savoir quelle fonction utiliser
* Il faut gérer les arguments des fonctions.

In [1]:
from fake_scpi import FakeSCPI
inst = FakeSCPI()

def set_frequency(inst, freq=1000):
    command = f'FREQ {freq:.3f}'
    inst.write(command)

def get_frequency(inst):
    command = f'FREQ?'
    return float(inst.query(command))

def get_model_identification(inst):
    res = inst.query("*IDN?")
    comp, model, serial, _ = res.split(',') 
    dic = {'compagnie':comp,
          'model':model,
          'serial':serial}
    return dic

set_frequency(inst, 100)
get_frequency(inst)
get_model_identification(inst)

{'compagnie': 'TEKTRONIX', 'model': 'DPO3014', 'serial': 'C012048'}

In [42]:
class Instrument(object):
    def __init__(self, conn):
        self._conn = conn
        
    def get_model_identification(self):
        res = self._conn.query("*IDN?")
        comp, model, serial, _ = res.split(',') 
        dic = {'compagnie':comp,
              'model':model,
              'serial':serial}
        return dic
    
    @property
    def idn(self):
        return self.get_model_identification()

    
class TektronixGBF(Instrument):
    def set_frequency(self, freq=1000):
        command = f'FREQ {freq:.3f}'
        self._conn.write(command)

    def get_frequency(self):
        command = f'FREQ?'
        return float(self._conn.query(command))
    
    frequency = property(get_frequency, set_frequency)
        
class AgilentGBF(Instrument):
    def set_frequency(self, freq=1000):
        command = f'FRE {freq:.3f}'
        self._conn.write(command)

    def get_frequency(self):
        command = f'FRE?'
        return float(self._conn.query(command))

gbf = TektronixGBF(FakeSCPI())

### Exercice : vecteur3D

### Exercice : livre et bibliographie

## Nouveaux concepts



### Méthodes spéciales

- `__init__`
- `__repr__`, `__str__`

Lorsque c'est possible, `__repr__` doit représenté au mieux l'objet. Souvent il s'agit d'une chaine de caractère qui en étant évaluée renvoie un objet similaire. La méthode `__str__` renvoie `__repr__` par défaut. Sinon, elle doit être plus simple.

Opérateur unaire et binaire

- `__neg__`
- `__add__`, `__sub__`, `__mul__`, `__truediv__`, `__mod__`, `__pow__`
- `__radd__`, ...
- `__eq__` (==), `__ne__` (!=), `__lt__` (<), `__le__` (<=), `__gt__`, `__ge__`
- `__or__` (|), `__and__` (&), `__xor__`(^)


Emulateur de contenu

- a[key] => `a.__getitem__(key)`
- a[key] = val => `a.__setitem__(key, val)`
- del a[key] => `a.__delitem__(key)`
- len(a) => `a.__len__()`
- for elm in a => `for elm in a.__iter__()`

### Attributs et property

- Attributs de class vs attributs d'objets
- property

### Héritage

- isinstance permet de tester si un objet est une instance d'une classe. 

In [5]:
class Test(object):
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
    def __getitem__(self, key):
        if key==0:
            return self.x
        if key==1:
            return self.y
        if key==2:
            return self.z
        raise ValueError
        
t = Test(1, 2, 3)
t[1]

2